In [1]:
!pip install transformers
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.utils import shuffle
import nltk
from tqdm import tqdm
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marinjaprincipe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marinjaprincipe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# this is copy from https://www.kaggle.com/code/soumyakushwaha/ethicalcommunicationai
# ----------------------------------------
stopword = set(stopwords.words('english'))

def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub(r"\@w+|\#",'',text)
    text = re.sub(r"[^\w\s]",'',text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    tweet_tokens = word_tokenize(text)
    filtered_tweets=[w for w in tweet_tokens if not w in stopword] #removing stopwords
    return " ".join(filtered_tweets)
#--------------------------------------------------------------------------------------

In [4]:

### second try
# oversample vom nur hate dataset, mit smooth maybe

# Constants
SEED = 42
BATCH_SIZE = 32
LEARNING_RATE = 1e-5
MAX_TEXT_LENGTH = 512
EPOCHS = 10
MODEL_PATH = 'roberta_model.bin'
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Set seeds
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)


labeled_data = pd.read_csv('./datasets/hate_speech_data.csv')
# Hate Speech and Offensive Language Data: 25.3k total entries.
# - Class 0: 1,430 entries (hate speech)
# - Class 1: 19,190 entries (offensive language)
# - Class 2: 4,163 entries (neither)

# Processing labeled hate speech dataset
hate_offensive_data = labeled_data[labeled_data['class'] != 2].copy()
hate_offensive_data.loc[:, 'category'] = hate_offensive_data['class'].replace([0, 1], 1)
hate_offensive_data = hate_offensive_data.rename(columns={'tweet': 'text'})

# Test 1 ---
# Select data for each class
hate_speech_data = labeled_data[labeled_data['class'] == 0].copy()
offensive_data = labeled_data[labeled_data['class'] == 1].copy()
neither_data = labeled_data[labeled_data['class'] == 2].copy()
sample_size = len(hate_speech_data)
offensive_sample = offensive_data.sample(n=sample_size, random_state=SEED)
neither_sample = neither_data.sample(n=sample_size, random_state=SEED)
hate_speech_data['category'] = 1
offensive_sample['category'] = 1
neither_sample['category'] = 0
sampled_data = pd.concat([hate_speech_data, offensive_sample, neither_sample], ignore_index=True)[['tweet', 'category']]
sampled_data.rename(columns={'tweet': 'text'}, inplace=True)
sampled_data['text'] = sampled_data['text'].apply(clean_text)  # Assuming clean_text is a defined function
train_data, intermediate_data = train_test_split(sampled_data, test_size=0.3, random_state=SEED)
validation_data, test_data = train_test_split(intermediate_data, test_size=0.5, random_state=SEED)
train_tokens = tokenizer(train_data['text'].tolist(), padding=True, truncation=True, max_length=MAX_TEXT_LENGTH, return_tensors='pt')
validation_tokens = tokenizer(validation_data['text'].tolist(), padding=True, truncation=True, max_length=MAX_TEXT_LENGTH, return_tensors='pt')
test_tokens = tokenizer(test_data['text'].tolist(), padding=True, truncation=True, max_length=MAX_TEXT_LENGTH, return_tensors='pt')
print(f"New Train data shape: {train_data.shape}")
print(f"New Validation data shape: {validation_data.shape}")
print(f"New Test data shape: {test_data.shape}")


New Train data shape: (3003, 2)
New Validation data shape: (643, 2)
New Test data shape: (644, 2)


In [8]:
sampled_data.head

<bound method NDFrame.head of                                                    text  category
0                            halelookyhere queer gaywad         1
1     hitethunduh alsarabsss hes beaner smh tell hes...         1
2     devilgrimz vigxrarts youre fucking gay blackli...         1
3     markroundtreejr lmfaoooo hate black people the...         1
4                      nochillpaz least im nigger lmfao         1
...                                                 ...       ...
4285  someone pushes hard today may surprised ho cap...         0
4286                          beanies cumming real soon         0
4287  rt teapot stuff broke loose must panic share g...         0
4288                            early bird catches worm         0
4289  cant wait till get full appetite back ate half...         0

[4290 rows x 2 columns]>

In [5]:
# Convert category column to integer type
hate_offensive_data['category'] = hate_offensive_data['category'].astype(int)

# Convert labels to tensors
train_labels_tensor = torch.tensor(train_data['category'].values, dtype=torch.long)
validation_labels_tensor = torch.tensor(validation_data['category'].values, dtype=torch.long)
test_labels_tensor = torch.tensor(test_data['category'].values, dtype=torch.long)

train_dataset = TensorDataset(train_tokens['input_ids'], train_tokens['attention_mask'], train_labels_tensor)
validation_dataset = TensorDataset(validation_tokens['input_ids'], validation_tokens['attention_mask'], validation_labels_tensor)
test_dataset = TensorDataset(test_tokens['input_ids'], test_tokens['attention_mask'], test_labels_tensor)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [6]:
def train_and_validate(train_dataloader, validation_dataloader, device, EPOCHS, loss_criterion, optimizer, roberta_model):
    # Initialize variables for early stopping and best model
    best_validation_loss = float('inf')  # set to infinity initially
    early_stopping_counter = 0  # counter for early stopping
    MAX_EARLY_STOPPING_COUNT = 2  # maximum number of epochs with increasing val_loss to wait

    # Training and Validation
    for epoch in range(EPOCHS):  # Use the EPOCHS constant defined earlier
        roberta_model.train()
        cumulative_train_loss = 0

        # Wrap train_dataloader in tqdm for progress bar
        for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}"):
            optimizer.zero_grad()
            input_ids, attention_mask, labels = [item.to(device) for item in batch]
            outputs = roberta_model(input_ids, attention_mask=attention_mask)[0]
            loss = loss_criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            cumulative_train_loss += loss.item()

        # Validation
        roberta_model.eval()
        cumulative_validation_loss = 0

        # Wrap validation_dataloader in tqdm for progress bar
        with torch.no_grad():
            for batch in tqdm(validation_dataloader, desc=f"Validating Epoch {epoch+1}"):
                input_ids, attention_mask, labels = [item.to(device) for item in batch]
                outputs = roberta_model(input_ids, attention_mask=attention_mask)[0]
                loss = loss_criterion(outputs, labels)
                cumulative_validation_loss += loss.item()

        average_validation_loss = cumulative_validation_loss / len(validation_dataloader)
        average_train_loss = cumulative_train_loss / len(train_dataloader)
        print(f"Epoch: {epoch+1}, Training Loss: {average_train_loss:.4f}, Validation Loss: {average_validation_loss:.4f}")

        # Check for early stopping
        if average_validation_loss < best_validation_loss:
            best_validation_loss = average_validation_loss
            torch.save(roberta_model.state_dict(), MODEL_PATH)
            early_stopping_counter = 0  # reset early stopping counter
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= MAX_EARLY_STOPPING_COUNT:
                print("Early stopping due to increasing validation loss.")
                break

# Load the Roberta model
roberta_model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2).to(device)
loss_criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.AdamW(params=roberta_model.parameters(), lr=LEARNING_RATE)

# Call the function with the desired data loaders
train_and_validate(train_dataloader, validation_dataloader, device, EPOCHS, loss_criterion, optimizer, roberta_model)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Validating Epoch 1: 100%|██████████| 21/21 [00:05<00:00,  3.71it/s]


Epoch: 1, Training Loss: 0.4207, Validation Loss: 0.2658


Validating Epoch 2: 100%|██████████| 21/21 [00:05<00:00,  3.88it/s]


Epoch: 2, Training Loss: 0.2067, Validation Loss: 0.2622


Validating Epoch 3: 100%|██████████| 21/21 [00:05<00:00,  3.92it/s]


Epoch: 3, Training Loss: 0.1625, Validation Loss: 0.2886


Validating Epoch 4: 100%|██████████| 21/21 [00:05<00:00,  3.90it/s]

Epoch: 4, Training Loss: 0.1313, Validation Loss: 0.3104
Early stopping due to increasing validation loss.


In [11]:
def test_model_on_data(model, data_loader, data_frame, batch_size, device):
    model.eval()  # Set the model to evaluation mode
    correct_predictions = 0
    total_predictions = 0
    misclassified_entries = []

    # Extract test sentences and labels from the dataframe
    test_sentences = data_frame['text'].tolist()
    test_labels = data_frame['category'].tolist()

    with torch.no_grad():
        for batch_num, batch in enumerate(tqdm(data_loader, desc='Testing', dynamic_ncols=True), 0): # Starting index at 0 for batch_num
            input_ids, attention_mask, labels = [item.to(device) for item in batch]
            outputs = model(input_ids, attention_mask=attention_mask)[0]
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

            # Identify misclassified sentences
            misclassifications = (predicted != labels).nonzero(as_tuple=True)[0].cpu().numpy()
            for idx in misclassifications:
                global_index = idx + batch_num * batch_size  # Calculate global index in test set
                misclassified_entries.append({
                    "index": global_index,
                    "sentence": test_sentences[global_index],
                    "true_label": 'offensive/hatespeech' if test_labels[global_index] == 1 else 'not offensive/hatespeech',
                    "predicted_label": 'offensive/hatespeech' if predicted[idx].item() == 1 else 'not offensive/hatespeech'
                })

    # Display test accuracy
    accuracy = 100 * correct_predictions / total_predictions
    print(f'Test Accuracy: {accuracy:.2f}%')

    # Display misclassified sentences
    print("\nMisclassified Sentences:")
    for entry in misclassified_entries:
        print(f"Index: {entry['index']}, Sentence: {entry['sentence']}, True Label: {entry['true_label']}, Predicted Label: {entry['predicted_label']}")



# test_loader old 99.35%
# test_dataloader_new new 92.24%
# remaining_dataloader old 99.31% new 96.46% (contains both sentiment and hate)
# hate_offensive_dataloader (only hate set)  old 99.50%  new 95.57%
# filtered_test_dataloader (spezialized hate dataset)  old 63.87% ouff  new 59.82%
test_model_on_data(roberta_model, test_dataloader, test_data, BATCH_SIZE, device)

Testing: 100%|██████████| 21/21 [00:01<00:00, 15.33it/s]

Test Accuracy: 92.24%

Misclassified Sentences:
Index: 10, Sentence: acmobbdeep madsbennett yeah run squirrels coons, True Label: not offensive/hatespeech, Predicted Label: offensive/hatespeech
Index: 21, Sentence: look little tell shes beautiful much itll make smile, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 39, Sentence: rt domhowx know youre ugly ape chosen man crush youve never, True Label: not offensive/hatespeech, Predicted Label: offensive/hatespeech
Index: 84, Sentence: , True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 99, Sentence: alex smith wears red yellow quit psycho ex boyfriends get hes past, True Label: not offensive/hatespeech, Predicted Label: offensive/hatespeech
Index: 100, Sentence: wan na play pin da tail da colored mans butt, True Label: not offensive/hatespeech, Predicted Label: offensive/hatespeech
Index: 102, Sentence: drfreezeburnn daithidenogla learned many retards find search trendi

In [ ]:
test_model_on_data(roberta_model, remaining_dataloader, remaining_data, BATCH_SIZE, device)

Testing: 100%|██████████| 4362/4362 [10:52<00:00,  6.69it/s]


Test Accuracy: 96.46%

Misclassified Sentences:
Index: 1, Sentence: asking supporters prefix chowkidar names modi great service confusion read crustal clear crass filthy nonsensical see abuses coming chowkidars, True Label: not offensive/hatespeech, Predicted Label: offensive/hatespeech
Index: 5, Sentence: gandhi gay modi, True Label: not offensive/hatespeech, Predicted Label: offensive/hatespeech
Index: 21, Sentence: born religion female deities worshipped misogynistic sadistic tradition totally point isits man made tradition written one religious lunatic support religion repressive, True Label: not offensive/hatespeech, Predicted Label: offensive/hatespeech
Index: 29, Sentence: hindustan seen worst hindus maj hindu rashtra thrashed rascal faces anti indian politiciansantinationals urban naxals wait watch modis win pakistan mein bhi hindu hona garv baat hogi, True Label: not offensive/hatespeech, Predicted Label: offensive/hatespeech
Index: 38, Sentence: itna fark, True Label: not off

In [ ]:
test_model_on_data(roberta_model, hate_offensive_dataloader, hate_offensive_data, BATCH_SIZE, device)

Testing: 100%|██████████| 638/638 [00:51<00:00, 12.41it/s]

Test Accuracy: 95.57%

Misclassified Sentences:
Index: 2, Sentence: rt cganderson vivabased look like tranny, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 25, Sentence: hoe hitting, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 96, Sentence: comedyposts harlem shake excuse go full retard seconds, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 143, Sentence: kingcuh io io alu record ho vine sai pe hahahaha lol anywaaaaaays haha, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 144, Sentence: vondreaam naw old tape new cover upcoming mixtape kingofthehill yeah im waiting hoe mayne, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 177, Sentence: kobe stay talking trash lets forget meant one smh, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 200, Sentence: samjlayman holy shit 

In [ ]:
test_model_on_data(roberta_model, test_dataloader, test_data, BATCH_SIZE, device)


Testing: 100%|██████████| 20/20 [00:01<00:00, 10.43it/s]

Test Accuracy: 95.75%

Misclassified Sentences:
Index: 10, Sentence: anitafarid angjly lmao side ho gets real action anyways, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 35, Sentence: rt dril ah smell ready opens oven pulls sshitty burnt ritz cracker perfect boys, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 97, Sentence: backbone every nicca need spine, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 129, Sentence: modi pakistan china centered, True Label: not offensive/hatespeech, Predicted Label: offensive/hatespeech
Index: 175, Sentence: absolutely right want modi sath hain, True Label: not offensive/hatespeech, Predicted Label: offensive/hatespeech
Index: 177, Sentence: djweezlove said want dj mix track regardless use video gon dumb retarded reg versi, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 221, Sentence: rt stfunobodycare, Tru

In [ ]:
test_model_on_data(roberta_model, filtered_test_dataloader, reduced_test_data, BATCH_SIZE, device)

Testing: 100%|██████████| 117/117 [00:04<00:00, 24.17it/s]


Test Accuracy: 59.82%

Misclassified Sentences:
Index: 6, Sentence: hate immigrants, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 18, Sentence: despise disabled people, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 20, Sentence: despise immigrants, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 27, Sentence: detest immigrants, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 32, Sentence: absolutely loathe disabled people, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 34, Sentence: absolutely loathe immigrants, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 39, Sentence: disgusted disabled people around, True Label: offensive/hatespeech, Predicted Label: not offensive/hatespeech
Index: 41, Sentence: disgusted immigrants around, True Label: offensive/hatespeech, Predicted 

In [ ]:
from sklearn.metrics import f1_score

roberta_model.eval()

accumulated_predictions = []
accumulated_labels = []

# Generate predictions for the filtered_test_dataloader
with torch.no_grad():
    for batch in tqdm(filtered_test_dataloader, desc="Computing predictions on test set"):
        input_ids, attention_mask, labels = [item.to(device) for item in batch]
        outputs = roberta_model(input_ids, attention_mask=attention_mask)[0]

        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        positive_class_predictions = probabilities[:, 1].cpu().numpy()  # get probabilities of the positive class
        accumulated_predictions.extend(positive_class_predictions)
        accumulated_labels.extend(labels.cpu().numpy())

# Grid Search for Optimal Threshold
optimal_threshold = 0.5  # default value
best_f1 = 0  # Initialized to 0 as we'll be maximizing F1 score
threshold_values = np.arange(0.1, 0.9, 0.01)

for threshold in threshold_values:
    thresholded_predictions = [1 if probability > threshold else 0 for probability in accumulated_predictions]
    current_f1 = f1_score(accumulated_labels, thresholded_predictions)
    if current_f1 > best_f1:
        best_f1 = current_f1
        optimal_threshold = threshold

print(f"Optimal threshold for the filtered test set: {optimal_threshold}")


Computing predictions on test set: 100%|██████████| 117/117 [00:03<00:00, 36.06it/s]


Optimal threshold for the filtered test set: 0.1
